In [1]:
# HIDDEN - sets path

import sys
sys.path.insert(1, '../..')

# Transfering views to TrendMiner
In this use case, views have been exported from another tool, and need to be transferred to TrendMiner. The tags used in these views are already connected and available in TrendMiner.


## Export format
The export view format is in the following simple (json) format:

In [2]:
view_data = [
    {
        "name": "Reactor overview",
        "description": "Overview of reactor tags",
        "tags": [
            {
                "name": "TM-BP2-CW.1",
                "color": "#d70c52"
            },
            {
                "name": "TM-BP2-LEVEL.1",
                "color": "#346121"
            },
            {
                "name": "TM-BP2-PRESSURE.1",
                "color": "#096096"
            }            
        ],
        "span": {
            "start": "2022-01-15 18:17:23",
            "end": "2022-01-17 12:23:12",
        },
        "stacked": True,
    }
]

## Creating the client
Setting the timezone is important as view timestamps will be assumed to be in this timezone when there is no timezone information present in the timestamp. We need to log in as the user for which the views need to be created. This user should have access to the datasources containing the tags in the views.

In [3]:
import keyring
from trendminer_interface import TrendMinerClient

url = "https://cs.trendminer.net"
client_id = "tmInterfaceClient"
username = "wdaniels"

client = TrendMinerClient(
    url = url,
    client_id = client_id,
    client_secret=keyring.get_password(url, client_id),
    username=username,
    password=keyring.get_password(url, username),
    tz="Europe/Brussels"
)

## View creation
Views in TrendMiner have more options than provided in the export format. By default, we choose to not show gridlines, filling under the tags, or context items.

In [4]:
chart_settings = {
    "grid": False,
    "filling": False,
    "context": False,
}

In TrendMiner, the timespan of the context bar (the small overview below the focus chart) is also a setting of the view. The timespan of the view should not necessarily be in this period, though it is cleaner if it is. Here, we choose the latest 2 years as the standard setting for all view, as the timespan of all views should fall in the latest 2 years.

In [5]:
context_interval="2y"

To keep our work organizer cleean, we will put the imported views together in a folder. The following command retrieves the folder if it already exists, and otherwise creates it.

In [6]:
folder = client.folder.from_path("Imports", create_new=True)

We can now iterate over the view data to initialize the views.

In [7]:
views = []

for data in view_data:
    
    # Load tags
    tags = []
    for tagdata in data["tags"]:
        tag = client.tag.from_name(tagdata["name"])
        tag.color = tagdata["color"]
        tags.append(tag)
        
    # Set view interval
    interval = client.interval(
        data["span"]["start"],
        data["span"]["end"]
    )
    
    # Chart configuration
    if data["stacked"]:
        chart = client.trend.chart.stacked(**chart_settings)
    else:
        chart = client.trend.chart.trend(**chart_settings)
        
    thv = client.trend.view(
        entries=tags,
        layers=[interval],
        chart=chart,
        context_interval=context_interval,
        folder=folder,
    )
    
    views.append(thv)

The views can now be saved on the server.

In [8]:
for view in views:
    view.post()

In [9]:
# HIDDEN - item cleanup
folder.delete()